## ChatBot UI using Gradio
create a chatbot UI using Gradio to interact with multiple LLMs (Gemma, Llama, Ministral) in a conversational manner.

In [3]:
import gradio as gr
from openai import OpenAI

In [4]:
!ollama serve

Error: listen tcp 127.0.0.1:11434: bind: Only one usage of each socket address (protocol/network address/port) is normally permitted.


In [5]:
# check if ollama server is running
import requests
requests.get("http://localhost:11434").content

b'Ollama is running'

In [6]:
ollama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [7]:
system_prompt = """You are an engaging and helpful AI assistant. Be conversational, warm, 
and show genuine curiosity and enthusiasm in your responses. response should be short and concise,"""

### To run using OpenAI API

In [8]:
def chat(user_message, chat_history, model_name):
    messages = ([{"role": "system", "content": system_prompt}] +
                chat_history +
                [{"role": "user", "content": user_message}])

    with ollama.chat.completions.stream(model=model_name, messages=messages) as stream:
        response = ""
        for chunk in stream:
            if chunk.type == "content.delta":
                response += chunk.delta or ""
            yield response

In [9]:
default_model = "llama3.2"
available_models = ["llama3.2", "gemma3", "ministral-3:3b"]

with gr.Blocks(theme=gr.themes.Soft(), title="Multi-Model Chatbot") as demo:
    gr.Markdown("# 🤖 Multi-Model Chatbot")
    gr.Markdown("Chat with different LLMs: Llama, Gemma, or Ministral")

    active_model = gr.State(default_model)
    active_model_text = gr.Markdown(f"**Active model:** `{default_model}`")

    with gr.Column():
        model_dropdown = gr.Dropdown(
            choices=available_models,
            value=default_model,
            show_label=False,
            container=True,
            info="Choose which LLM to chat with"
        )
        use_model_btn = gr.Button("Switch Model", variant="primary", size="sm")

    def apply_model(model_name: str):
        return model_name, f"**✓ Active model:** `{model_name}`"

    use_model_btn.click(
        fn=apply_model,
        inputs=model_dropdown,
        outputs=[active_model, active_model_text],
    )

    gr.ChatInterface(
        fn=chat,
        additional_inputs=[active_model],
        title="",
        description="",
    )

demo.launch(show_error=True, share=False)

C:\Users\Priyanshu\AppData\Local\Temp\ipykernel_17708\479627450.py:4: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), title="Multi-Model Chatbot") as demo:


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
